## Extractable

In [4]:
import extractable as ex

In [ ]:
input_file = "../data/NATS-Nursery-Ltd-Catalogue-2019_NativePerennials.pdf"
output_dir = "../data/NativePerennials/"

ex.extract(input_file=input_file, output_dir=output_dir, output_filetype=ex.Filetype.CSV)

## PyMuPDF

Reference: https://github.com/pymupdf/PyMuPDF-Utilities/blob/master/table-analysis/join_tables.ipynb

In [1]:
import pandas as pd
import os
import fitz
if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")

from config import columns, config, corrections, mappings

pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_rows", 10)
pd.set_option("max_colwidth", 15)

In [2]:
input_file_prefix = config.PDF_PREFIX
output_dir = config.CATALOG_DIR
output_file = config.CATALOG_FILE_NATS
# output_file_modified = os.path.join(output_dir, "catalogue_nats_merged_modified.csv")


def replace_acronyms(column, mapping):
    for acronym, full_text in mapping.items():
        column = column.str.replace(r'\b' + acronym + r'\b', full_text, regex=True)
    return column


def extract_category_data(category, category_name):
    input_file = f'{input_file_prefix}_{category}.pdf'
    doc = fitz.open(input_file)
    
    print(f'=== {category} ===')
    df_fragments = []  # list of DataFrames per table fragment
    for page in doc:  # iterate over the pages
        tabs = page.find_tables()
        if len(tabs.tables) == []:
            break
        tab = tabs[0]  # the 1st table
        df_tmp = tab.to_pandas()
        df_fragments.append(df_tmp)  # append this DataFrame
        print(df_tmp.shape)
    
    df = pd.concat(df_fragments)
    df['CATEGORY'] = category_name
    
    df.replace({r"‘|’": "'", r"“|”": '"', "\n": " "}, regex=True, inplace=True)

    df['SUN'] = replace_acronyms(df['SUN'], mappings.SUN_MAPPING)
    df['SOIL'] = replace_acronyms(df['SOIL'], mappings.SOIL_MAPPING)
    df['HABITAT'] = replace_acronyms(df['HABITAT'], mappings.HABITAT_MAPPING)
    try:
        df['SPEC SIZES'] = replace_acronyms(df['SPEC SIZES'], mappings.SIZE_MAPPING)
    except KeyError:
        pass

    return df

In [3]:
os.makedirs(output_dir, exist_ok=True)

for category, category_name in mappings.CATEGORIES.items():
    df = extract_category_data(category, category_name)
    
    output_file_category = os.path.join(output_dir, f'{category}.csv')
    df.to_csv(output_file_category, index=False)
    print(df, end='\n\n')
    print(f'File saved: {output_file_category}', end='\n\n')

=== NativePerennials ===
(19, 6)
(25, 6)
(25, 6)
(25, 6)
(3, 6)
             LATIN          COMMON             SUN            SOIL         HABITAT      SPEC SIZES        CATEGORY
0   Achillea mi...          Yarrow  Sun/Partial...  Moist/Dry, ...  Disturbed S...  Plug, Conta...  Native Pere...
1   Achlys trip...    Vanilla Leaf  Shade/Parti...    Moist, Humus  Shade Fores...       Container  Native Pere...
2   Allium acum...  Hooker's Onion             Sun  Well Draine...  Grassland, ...  Plug, Conta...  Native Pere...
3   Allium cernuum   Nodding Onion  Sun/Partial...  Well Draine...  Shoreline, ...  Plug, Conta...  Native Pere...
4   Allium scho...          Chives  Sun/Partial...    Well Drained  Disturbed S...  Plug, Conta...  Native Pere...
..             ...             ...             ...             ...             ...             ...             ...
23  Vancouveria...  Inside-out ...   Partial Shade  Moist/Dry, ...    Shade Forest  Plug, Conta...  Native Pere...
24    Viola adun

In [4]:
dfs = []
for category in mappings.CATEGORIES:
    df = pd.read_csv(os.path.join(output_dir, f'{category}.csv'))
    
    # Rename 'SPEC SIZES' to 'SIZES'
    if 'SPEC SIZES' in df.columns:
        df.rename(columns={'SPEC SIZES': 'SIZES'}, inplace=True)
    
    dfs.append(df)

df_merged = pd.concat(dfs, ignore_index=True)
col_names_original = df_merged.columns

# Create a new column to store original values, set to '' for non-changed values
df_merged['LATIN_0'] = df_merged['LATIN'].where(df_merged['LATIN'].isin(corrections.LATIN_REPLACE_NATS.keys()), '')
df_merged['COMMON_0'] = df_merged['COMMON'].where(df_merged['COMMON'].isin(corrections.COMMON_REPLACE_NATS.keys()), '')
# Correct names
df_merged['LATIN'] = df_merged['LATIN'].replace(corrections.LATIN_REPLACE_NATS)
df_merged['COMMON'] = df_merged['COMMON'].replace(corrections.COMMON_REPLACE_NATS)

df_merged.sort_values(by='LATIN', inplace=True)
df_merged = df_merged[columns.COL_NAMES_NATS]

df_merged.to_csv(output_file, index=False)
print(df_merged.shape, end='\n\n')
print(f'File saved: {output_file}', end='\n\n')

# for col in columns.COL_NAMES_ALL:
#     if col not in col_names_original:
#         df_merged[col] = '' 

# df_merged = df_merged[columns.COL_NAMES_ALL]

# df_merged.to_csv(output_file_modified, index=False)
# print(df_merged)
# print(f'File saved: {output_file_modified}')

(359, 7)

File saved: ./catalogues/catalogue_merged_original.csv

    IN_STOCK           LATIN          COMMON SQUAMISH HALKOMELEM FAMILY        CATEGORY KEYWORDS MAX_HEIGHT             SUN            SOIL         HABITAT           SIZES GROWING_EASE USE_VALUE ATTRACTS URL IMAGE DESCRIPTION
0             Achillea mi...          Yarrow                             Native Pere...                      Sun/Partial...  Moist/Dry, ...  Disturbed S...  Plug, Conta...                                                      
1             Achlys trip...    Vanilla Leaf                             Native Pere...                      Shade/Parti...    Moist, Humus  Shade Fores...       Container                                                      
2             Allium acum...  Hooker's Onion                             Native Pere...                                 Sun  Well Draine...  Grassland, ...  Plug, Conta...                                                      
3             Allium cernuum  